#### CONDICTION MONITORING OF SINGLE PHASE INDUCTION MOTOR USING MOTION AMPLIFICATION OF VIDEO

#### Preparations:


In [ ]:
!pip install --quiet -r requirements.txt
!pip install --quiet gdown mediapy

#### Download and load the well-trained weights:

In [ ]:
!wget https://github.com/ZhengPeng7/motion_magnification_learning-based/releases/download/v1.0/magnet_epoch12_loss7.28e-02.pth

In [ ]:
from magnet import MagNet
from callbacks import gen_state_dict
from config import Config


# config
config = Config()
# Load weights
weights_path = 'magnet_epoch12_loss7.28e-02.pth'
ep = int(weights_path.split('epoch')[-1].split('_')[0])
state_dict = gen_state_dict(weights_path)

model_test = MagNet().cuda()
model_test.load_state_dict(state_dict)
model_test.eval()
print("Loading weights:", weights_path)

#### Preprocessing

In [ ]:
# Turn the video into frames and make them into frame_ACB format.
file_to_be_maged = 'motor.avi'
video_name = file_to_be_maged.split('.')[0]
video_format = '.' + file_to_be_maged.split('.')[-1]


sh_file = 'VIDEO_NAME={}\nVIDEO_FORMAT={}'.format(video_name, video_format) + """


mkdir ${VIDEO_NAME}
ffmpeg -i ${VIDEO_NAME}${VIDEO_FORMAT} -f image2 ${VIDEO_NAME}/%06d.png
python make_frameACB.py ${VIDEO_NAME}
mkdir test_dir
mv ${VIDEO_NAME} test_dir
"""
with open('test_preproc.sh', 'w') as file:
  file.write(sh_file)

!bash test_preproc.sh

In [ ]:
import os
import sys
import cv2
import torch
import numpy as np
from data import get_gen_ABC, unit_postprocessing, numpy2cuda, resize2d



for testset in [video_name]:
    dir_results = 'res_' + testset
    if not os.path.exists(dir_results):
        os.makedirs(dir_results)

    config.data_dir = 'test_dir'
    data_loader = get_gen_ABC(config, mode='test_on_'+testset)
    print('Number of test image couples:', data_loader.data_len)
    vid_size = cv2.imread(data_loader.paths[0]).shape[:2][::-1]

    # Test
    for amp in [10, 25, 50]:
        frames = []
        data_loader = get_gen_ABC(config, mode='test_on_'+testset)
        for idx_load in range(0, data_loader.data_len, data_loader.batch_size):
            if (idx_load+1) % 100 == 0:
                print('{}'.format(idx_load+1), end=', ')
            batch_A, batch_B = data_loader.gen_test()
            amp_factor = numpy2cuda(amp)
            for _ in range(len(batch_A.shape) - len(amp_factor.shape)):
                amp_factor = amp_factor.unsqueeze(-1)
            with torch.no_grad():
                y_hats = model_test(batch_A, batch_B, 0, 0, amp_factor, mode='evaluate')
            for y_hat in y_hats:
                y_hat = unit_postprocessing(y_hat, vid_size=vid_size)
                frames.append(y_hat)
                if len(frames) >= data_loader.data_len:
                    break
            if len(frames) >= data_loader.data_len:
                break
        data_loader = get_gen_ABC(config, mode='test_on_'+testset)
        frames = [unit_postprocessing(data_loader.gen_test()[0], vid_size=vid_size)] + frames

        # Make videos of framesMag
        video_dir = os.path.join(dir_results, testset)
        if not os.path.exists(video_dir):
            os.makedirs(video_dir)
        FPS = 30
        video_save_path = os.path.join(video_dir, '{}_amp{}{}'.format(testset, amp, video_format))
        out = cv2.VideoWriter(
            video_save_path,
            cv2.VideoWriter_fourcc(*'DIVX'),
            FPS, frames[0].shape[-2::-1]
        )
        for frame in frames:
            frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
            cv2.putText(frame, 'amp_factor={}'.format(amp), (7, 37),
                        fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(0, 0, 255), thickness=2)
            out.write(frame)
        out.release()
        print('{} has been done.'.format(video_save_path))